In [ ]:
from google.colab import drive    
 
drive.mount('/content/drive')
test_path='/content/drive/MyDrive/test_images'
train_path='/content/drive/MyDrive/train_images'
train_cpath='/content/drive/MyDrive/train.csv'
test_cpath='/content/drive/MyDrive/test.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
dataset = pd.read_csv(train_cpath)
dataset.values.tolist()
L=dataset['ClassName']
L1=dataset['ImageId']
l=np.unique(np.array(L))
l1=np.unique(np.array(L1))
img={}
cl={}
for i in range(len(l)):
  cl[i]=l[i]
cls_k=list(cl.keys())
cls_v=list(cl.values())
for i in range(len(dataset)):
  img[L1[i]]=cls_k[cls_v.index(L[i])]

In [ ]:
import os
import cv2
dataset = pd.read_csv(test_cpath)
dataset.values.tolist()
L=dataset['ImageId']
l=list(np.unique(np.array(L)))
X_test=[]
for i in range(len(dataset)):
  path=os.path.join(test_path,l[i])
  image=cv2.imread(path,0)
  image=cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
  X_test.append((cv2.resize(image,(150,150))))

In [ ]:
X=[]
Y=[]
import cv2
import os
d=0
for a,b in img.items():
  d=d+1
  path=os.path.join(train_path,a)
  d
  image=cv2.imread(path,0)
  image=cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
  X.append(cv2.resize(image,(150,150)))
  Y.append(b)

In [ ]:
import tensorflow.keras as tfk
from keras.applications.resnet50 import ResNet50
x = tfk.applications.resnet50.preprocess_input(np.array(X[0:5001]))
y = tfk.utils.to_categorical(Y[0:5001], 61)
X_test = tfk.applications.resnet50.preprocess_input(np.array(X_test))


baseModel = tfk.applications.ResNet50(include_top=False, weights="imagenet", input_tensor= tfk.Input(shape=(150, 150, 3)))
for layer in baseModel.layers[:143]:
  layer.trainable = False
from sklearn.model_selection import train_test_split

X_train,X_t,Y_train,Y_t=train_test_split(x,y)
X_train=np.asarray(X_train,float)/256
Y_train=np.asarray(Y_train)
X_t=np.asarray(X_t,float)/256
Y_t=np.asarray(Y_t)
from keras.utils import to_categorical
from tensorflow.keras import layers
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout
from keras.models import Sequential
from tensorflow.keras import activations
from tensorflow.keras import regularizers
model= Sequential()
model.add(Conv2D(kernel_size=(3,3), filters=32, activation='relu', input_shape=(150,150,3),activity_regularizer=regularizers.L2(0.01)))
model.add(Dense(20,activation='relu'))
model.add(layers.Activation(activations.relu))
model.add(Conv2D(kernel_size = (3,3),filters=64,activation='relu',activity_regularizer=regularizers.L2(0.01)))
model.add(MaxPool2D(pool_size=(2,2),strides=(1, 1), padding='valid'))
model.add(Conv2D(kernel_size=(3,3),filters=128,activation='relu',activity_regularizer=regularizers.L2(0.01)))
model.add(MaxPool2D(pool_size=(2,2),strides=(1, 1), padding='valid'))
model.add(Conv2D(kernel_size=(3,3),filters=256,activation='relu',activity_regularizer=regularizers.L2(0.01)))
model.add(Flatten())
model.add(Dense(61,activation = 'softmax'))
    
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='SGD')


model.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1,validation_data=(X_t, Y_t))


Epoch 1/50


In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_Test, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred_bool_cat = tfk.utils.to_categorical(y_pred_bool, 61)
print(classification_report(y_test, y_pred_bool_cat))
test_dataset = pd.read_csv(testDataPath)
finalLabels = test_dataset.copy()
predLabels = []
for val in y_pred_bool:
  predLabels.append(train_classes[val])
finalLabels['ClassName'] = pd.Series(predLabels, index=finalLabels.index)
finalLabels.to_csv('submit.csv', header=True, index=False)
